## RKR Computation
All Code and Markdown written by Gary Zeri, Chapman University Student and member of the LaRue Cat Lab

<br> <br>

All equations and information within this notebook originated from <i>The Computation of RKR Potential Energy Curves of Diatomic Molecules using Matematica</i>, written by Peter Senn.

<br>
The RKR method is a procedure used to determine the potential energy curves of molecules by calculating the classical turning points, $r_-$ and $r_+$ from Diatomic Constants of molecules.

<br>

The turning points, $r_{\pm}$, can be computed using the following equation:
$$r_{\pm} = \frac{f(v)}{2} \cdot \left[\sqrt{1 + \frac{1}{f(v)g(v)}} \pm 1\right]$$
<br>
It should be noted that in the above equation the fraction in the square root has a one in the numerator while in the paper by Peter Senn the numerator has a four. The four is not present in the Matematica code in Senn's paper and in <i>Lecture #21: Construction of Potential Curves by the Rydberg-Klein-Rees Method (RKR)</i> from MIT OpenCourseWare, the four is not included either, leading to the omission of the four in the equation presented here to provide the closest possible equation to that useed within the code.
<br>
Where $f(v)$ and $g(v)$ are defined as follows: 
$$f(v) = \int^v_{\frac{-1}{2}}{ \left[ \frac{dv^{'}}{\sqrt{E(v)-E(v^{'})}{}} \right] }$$
<br>
$$g(v) = \int_{\frac{-1}{2}}^v{\left[ \frac{\frac{\partial E}{\partial J}}{\sqrt{E(v) - E(v^{'})}}\right]dv^{'}}$$

<br>

In the original paper by Senn, there are constants in front of the integrals that are omitted here as they are constants used to scale the values to a desired unit system. Since this RKR code will use wavenumbers for energy and non-Hartree atomic units for distance, the constant of $\frac{8.211609}{\sqrt{\mu}}$ will be multiplied by the final answers to perform the conversion to hartree atomic units.

<br>

For vibronic states with no rotation and $J=0$, $E(v)$ and $\frac{\partial E}{\partial J}$ can be represented as:
$$E(v) = \omega_e(v+ \frac{1}{2}) - \omega_ex_e(v+\frac{1}{2})^2 + \omega_ey_e(v+\frac{1}{2})^3 + \omega_ez_e(v+\frac{1}{2})^4 + \dots$$
<br>
$$\frac{\partial E}{\partial J} = B(v) = B_e -\alpha_e(v+\frac{1}{2} + y_e(v+\frac{1}{2})^2 + \dots$$

<br> 

An important computational issue to note with the RKR method is that the integrand term $\frac{dv^{'}}{\sqrt{E(v) - E(v^{'})}}$ will become one over zero since the integration is from $\frac{-1}{2}$ to $v$, thus the integrand will become $\frac{1}{\sqrt{E(v) - E(v)}}$ when $v^{'} = v$ at the end of the integration.<br>
In order to deal with this issue, the integral is changed from $\int^{v}_{\frac{-1}{2}}$ to $\int^{v-\delta}_{\frac{-1}{2}}$, where $\delta$ is some extremely small value. A correction term is then added to both $f(v)$ and $g(v)$ to account for the missing endpoint of the integration resulting in new and more easily computable versions of $f(v)$ and $g(v)$:

$$f(v) = \int^{v-\delta}_{\frac{-1}{2}}{\frac{dv^{'}}{\sqrt{E(v) - E(v^{'})}} + \sqrt{\frac{\delta}{Q_v}}}$$
<br>
$$g(v) = \int^{v-\delta}_{\frac{-1}{2}}{\frac{B(v^{'})}{\sqrt{E(v)-E(v^{'})}}dv^{'} } + 2B(v)\sqrt{\frac{\delta}{Q_v}} $$
<br>
Where $Q_v$ is the following series:
$$Q_v = w_e-2\omega_ex_e(v+\frac{1}{2}) + 3\omega_ey_e(v+\frac{1}{2})^2 + 4\omega_ez_e(v+\frac{1}{2})^3 + \dots$$

<br>

The following code only uses the terms listed above in the computation for all series listed.

In [38]:
#Import all needed classes here and create all global variables

import math
import numpy as np
from scipy.integrate import quad as integrate

from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

#Should be a very small value 
#Is the distance from v that the integration stops at
delta = pow(10, -5)

#Reduced Molecular Mass
#In Non Hartree Atomic Units, each proton has an amu of 1
u = 0.5

#Diatomic Constants, must be in Wavenumbers (1 / cm)
we = 4401.21
wxe = 121.33
wye = 0
wze = 0
Be = 60.853
alphae = 3.062
ye = 0

In [2]:
#Define All Functions Here

def E(v):
    term = v + 0.5
    return (we * term) - (wxe*pow(term, 2)) + (wye*pow(term, 3)) + (wze*pow(term,4))

def B(v):
    term = v + 0.5
    return Be - (alphae * term)  + ye*pow(term, 2)

#Used in the correctionFactor calculation
def Q(v):
    term = v + 0.5
    return we - (2*wxe*term) + (3*wye*pow(term, 2)) + (4*wze*pow(term,3))

#Used to correct integrals that stop delta away from v
def correctionFactor(v):
    return 2 * math.sqrt(delta / Q(v))

def integralRadical(v, vPrime):        
    return math.sqrt(E(v) - E(vPrime))

def f(v):
    integrand = lambda vPrime: 1 / integralRadical(v, vPrime)
    return integrate(integrand, -0.5, v-delta)[0] + correctionFactor(v)

def g(v):
    integrand = lambda vPrime : B(vPrime) / integralRadical(v, vPrime)
    return integrate(integrand, -0.5, v-delta)[0] + (B(v)*correctionFactor(v))
    
#v refers to a float energy level for the potential well
#returns the tuple (r+, r-)
def RKR(v):

    c0 = (8.211609 * f(v) ) / (2 * math.sqrt(u))

    radicand = 1 / (f(v) * g(v))     
    c1 = math.sqrt(1 + radicand)

    return c0 * (c1 + 1), c0 * (c1 - 1)

In [44]:
#Lists to hold data that will be graphed
x = []
y = []

#Loop to generate all data to graph
for v in np.arange(-.499, 12, .01):
    
    x.extend( RKR(v) )
    y.extend( [E(v)] * 2 ) 
    
#Prepare data for graphing
figure = {
    
    "data":[
        {
            "type":"scatter",
            "x":x,
            "y":y,
            "connectgaps":True,
            "mode":"markers",
            "name":"RKR Potential",
        }
    ],
    
    "layout":{
        "xaxis":{"title":"Bond Distance non-Hartree Atomic Units"},
        "yaxis":{"title":"Energy in Wavenumbers"},
        "title":{"text":"RKR Generated Potential Energy Curve"}
    }
}

iplot(figure)

-0.49999
-0.48999
-0.47998999999999997
-0.46998999999999996
-0.45998999999999995
-0.44998999999999995
-0.43998999999999994
-0.42998999999999993
-0.4199899999999999
-0.4099899999999999
-0.3999899999999999
-0.3899899999999999
-0.3799899999999999
-0.3699899999999999
-0.35998999999999987
-0.34998999999999986
-0.33998999999999985
-0.32998999999999984
-0.31998999999999983
-0.3099899999999998
-0.2999899999999998
-0.2899899999999998
-0.2799899999999998
-0.2699899999999998
-0.2599899999999998
-0.24998999999999977
-0.23998999999999976
-0.22998999999999975
-0.21998999999999974
-0.20998999999999973
-0.19998999999999972
-0.18998999999999971
-0.1799899999999997
-0.1699899999999997
-0.1599899999999997
-0.14998999999999968
-0.13998999999999967
-0.12998999999999966
-0.11998999999999965
-0.10998999999999964
-0.09998999999999963
-0.08998999999999963
-0.07998999999999962
-0.06998999999999961
-0.0599899999999996
-0.04998999999999959
-0.03998999999999958
-0.029989999999999573
-0.019989999999999564
-0.009989

3.850010000000004
3.860010000000004
3.8700100000000037
3.8800100000000044
3.890010000000004
3.900010000000004
3.9100100000000038
3.9200100000000035
3.9300100000000042
3.940010000000004
3.950010000000004
3.9600100000000045
3.9700100000000043
3.980010000000004
3.990010000000004
4.000010000000003
4.010010000000004
4.020010000000004
4.030010000000003
4.040010000000004
4.050010000000004
4.060010000000004
4.0700100000000035
4.080010000000003
4.090010000000004
4.100010000000004
4.1100100000000035
4.120010000000004
4.130010000000004
4.140010000000004
4.1500100000000035
4.160010000000003
4.170010000000004
4.180010000000004
4.190010000000004
4.200010000000004
4.210010000000004
4.220010000000004
4.230010000000004
4.240010000000003
4.250010000000004
4.260010000000004
4.270010000000004
4.280010000000004
4.290010000000004
4.300010000000004
4.310010000000004
4.3200100000000035
4.330010000000004
4.340010000000004
4.350010000000004
4.360010000000004
4.370010000000004
4.380010000000004
4.390010000000004

8.420010000000008
8.430010000000008
8.440010000000008
8.450010000000008
8.460010000000008
8.470010000000007
8.480010000000007
8.490010000000007
8.500010000000007
8.510010000000008
8.520010000000008
8.530010000000008
8.540010000000008
8.550010000000007
8.560010000000007
8.570010000000007
8.580010000000009
8.590010000000008
8.600010000000008
8.610010000000008
8.620010000000008
8.630010000000008
8.640010000000007
8.650010000000007
8.660010000000007
8.670010000000008
8.680010000000008
8.690010000000008
8.700010000000008
8.710010000000008
8.720010000000007
8.730010000000007
8.740010000000009
8.750010000000009
8.760010000000008
8.770010000000008
8.780010000000008
8.790010000000008
8.800010000000007
8.810010000000007
8.820010000000007
8.830010000000009
8.840010000000008
8.850010000000008
8.860010000000008
8.870010000000008
8.880010000000008
8.890010000000007
8.900010000000009
8.910010000000009
8.920010000000008
8.930010000000008
8.940010000000008
8.950010000000008
8.960010000000008
8.97001000